In [ ]:
# 먼저 업로드된 파일을 확인하여 데이터 내용을 살펴보겠습니다.
import pandas as pd

# 파일 경로
file_path = '정신질환 사망률.csv'

# 파일 로드
data = pd.read_csv(file_path)
data.head()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# ARIMA 모델 학습 및 예측
def arima_forecast(data, order, forecast_steps):
    model = ARIMA(data, order=order)
    result = model.fit()
    forecast = result.get_forecast(steps=forecast_steps)
    forecast_index = pd.date_range(start=data.index[-1] + pd.DateOffset(years=1), periods=forecast_steps, freq='Y')
    forecast_series = pd.Series(forecast.predicted_mean.values, index=forecast_index)
    return forecast_series, result

# ARIMA 모델 설정 (p=1, d=1, q=1)
order = (1, 1, 1)
forecast_steps = 50  # 향후 50년 예측

# 모델 학습 및 예측
forecast, model_result = arima_forecast(add_up_data["Mental"], order, forecast_steps)

# 결과 요약
print(model_result.summary())

# 예측 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(add_up_data, label="Observed")
plt.plot(forecast, label="Forecast", color="orange")
plt.title("Mental: Observed vs Forecast (ARIMA)")
plt.xlabel("Year")
plt.ylabel("Mental")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# SEJONG 데이터 추출 및 전처리
sejong_data = data[data["AREA"] == "SEJONG"].iloc[:, 1:]  # "AREA" 제외
sejong_data = sejong_data.T  # 전치
sejong_data.columns = ["Mental"]  # 열 이름 설정

# 2012년 이후 데이터 사용
sejong_data_cleaned = sejong_data.loc["2012":].dropna()

# 데이터 확인
sejong_data_cleaned.head()


In [ ]:
from scipy.optimize import curve_fit

# 비선형 함수 정의 (예: 지수 함수)
def nonlinear_model(x, a, b, c):
    return a * np.exp(b * (x - 2012)) + c

# 데이터 준비
X_train_nonlinear = pd.to_datetime(sejong_data_cleaned.index).year.values  # Convert index to DatetimeIndex and extract year
y_train_nonlinear = sejong_data_cleaned["Mental"].values

# 비선형 회귀 모델 학습
popt, pcov = curve_fit(nonlinear_model, X_train_nonlinear, y_train_nonlinear, maxfev=10000)

# 2007~2011년 데이터 생성 및 예측
years_to_predict_nonlinear = np.arange(2007, 2012)
predicted_counts_nonlinear = nonlinear_model(years_to_predict_nonlinear, *popt)

# 예측 결과 시각화
plt.figure(figsize=(12, 6))
plt.scatter(X_train_nonlinear, y_train_nonlinear, label="Observed (2012-2023)", color="blue")
plt.plot(years_to_predict_nonlinear, predicted_counts_nonlinear, label="Predicted (2007-2011, Nonlinear)", color="orange")
plt.title("SEJONG Mental: Nonlinear Regression Prediction")
plt.xlabel("Year")
plt.ylabel("Mental")
plt.legend()
plt.grid()
plt.show()

# 예측 결과 반환
predicted_counts_nonlinear